In [14]:
import numpy as np
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/iiit hackathon/UNSW_NB15_training-set.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               82332 non-null  int64  
 1   dur              82332 non-null  float64
 2   proto            82332 non-null  object 
 3   service          82332 non-null  object 
 4   spkts            82332 non-null  int64  
 5   dpkts            82332 non-null  int64  
 6   sbytes           82332 non-null  int64  
 7   dbytes           82332 non-null  int64  
 8   sinpkt           82332 non-null  float64
 9   dinpkt           82332 non-null  float64
 10  tcprtt           82332 non-null  float64
 11  synack           82332 non-null  float64
 12  ackdat           82332 non-null  float64
 13  smean            82332 non-null  int64  
 14  dmean            82332 non-null  int64  
 15  is_sm_ips_ports  82332 non-null  int64  
 16  attack_cat       82332 non-null  object 
 17  label       

In [8]:
df.attack_cat.value_counts()

attack_cat
Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
Name: count, dtype: int64

In [11]:
pip install chart_studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.9 MB/s eta 0:00:00


In [40]:
print('zinger balls')

zinger balls


In [24]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import TensorDataset
# Step 2: Preprocess the data
# Separate numerical and label columns
numerical_columns = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'sinpkt', 'dinpkt', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'is_sm_ips_ports', 'label']  # List your numerical column names here
label_columns = ['proto', 'service', 'attack_cat']  # List your label column names here

# Preprocess numerical columns (scaling)
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Preprocess label columns (encoding)
label_encoders = {}
for col in label_columns:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

# Step 3: Convert to tensors
# Convert numerical columns to tensor
numerical_data = torch.tensor(df[numerical_columns].values, dtype=torch.float32)

# Convert label columns to tensor
label_tensors = [torch.tensor(df[col].values, dtype=torch.long) for col in label_columns]

# Combine numerical and label tensors if needed
# For example, if you want to have one tensor for all labels
# Reshape labels_combined to match the number of samples in numerical_data


# You can concatenate them along a certain dimension
# Assuming label tensors are 1D tensors
labels_combined = torch.cat(label_tensors, dim=0)
labels_combined = labels_combined.view(-1, 3)  # Assuming each sample has one label

# Create a PyTorch TensorDataset
dataset = TensorDataset(numerical_data, labels_combined)

# Now you have your dataset ready as a PyTorch TensorDataset


In [25]:
print(numerical_data.shape)
print(labels_combined.shape)


torch.Size([82332, 14])
torch.Size([82332, 3])


In [37]:
import torch.nn.functional as F

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Step 1: Define your LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Forward pass through LSTM
        out, _ = self.lstm(x)
        # Reshape inputs to add a sequence length dimension of 1
        # Apply linear layer to the last output of the sequence
        out = self.fc(out[:, -1, :])  # Select the last output of each sequence
        return out



# Step 2: Prepare your dataset
# Assuming you have numerical_data and labels_combined tensors
dataset = TensorDataset(numerical_data, labels_combined)

# Step 3: Define your data loaders
batch_size = 64
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Step 4: Define your loss function and optimizer
model = LSTMClassifier(input_size=14, hidden_size=128, num_layers=2, num_classes=3)  # Adjust parameters as needed
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: Train your model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
      optimizer.zero_grad()
      inputs = inputs.unsqueeze(1)
      outputs = model(inputs)

    # Apply softmax activation to convert logits to probabilities
      outputs = F.softmax(outputs, dim=1)
      labels = labels.long()
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()



# Inside your training loop



    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Validation Loss: {val_loss/len(val_loader):.4f}, '
              f'Validation Accuracy: {(100*correct/total):.2f}%')

# Step 6: Evaluate your model on the test dataset if available
# Evaluate your model similarly to the validation loop on a separate test dataset

# Optionally, save your model
#torch.save(model.state_dict(), 'lstm_model.pth')


RuntimeError: Expected floating point type for target with class probabilities, got Long

In [57]:
for inputs, labels in train_loader:
    optimizer.zero_grad()
    inputs = inputs.unsqueeze(1)
    outputs = model(inputs)

    # Apply softmax activation to convert logits to probabilities
    outputs = F.softmax(outputs, dim=1)

    # Debugging: Print outputs and labels
    print("Outputs:", outputs)
    print("Labels:", labels)

    # Ensure labels are of type torch.long
    labels = labels.long()

    # Compute the loss
    loss = criterion(outputs, labels)

    # Backpropagation and optimization
    loss.backward()
    optimizer.step()


Outputs: tensor([[0.3031, 0.3478, 0.3491],
        [0.3036, 0.3445, 0.3519],
        [0.3034, 0.3448, 0.3518],
        [0.3034, 0.3452, 0.3514],
        [0.3036, 0.3448, 0.3516],
        [0.3028, 0.3452, 0.3521],
        [0.3036, 0.3446, 0.3518],
        [0.3034, 0.3448, 0.3518],
        [0.3037, 0.3457, 0.3506],
        [0.3036, 0.3446, 0.3518],
        [0.3028, 0.3454, 0.3518],
        [0.3036, 0.3448, 0.3516],
        [0.3036, 0.3451, 0.3513],
        [0.3036, 0.3452, 0.3512],
        [0.3036, 0.3446, 0.3518],
        [0.3029, 0.3450, 0.3521],
        [0.3027, 0.3453, 0.3521],
        [0.3045, 0.3452, 0.3503],
        [0.3054, 0.3456, 0.3490],
        [0.3036, 0.3446, 0.3518],
        [0.3098, 0.3434, 0.3468],
        [0.3035, 0.3452, 0.3513],
        [0.3036, 0.3445, 0.3519],
        [0.3033, 0.3453, 0.3514],
        [0.3036, 0.3446, 0.3518],
        [0.3036, 0.3446, 0.3518],
        [0.3007, 0.3504, 0.3490],
        [0.3036, 0.3460, 0.3503],
        [0.3036, 0.3454, 0.3509],
     

RuntimeError: Expected floating point type for target with class probabilities, got Long

In [48]:
labels.dtype

torch.int64

In [49]:
outputs.dtype

torch.float32